In [ ]:
from datetime import datetime,timedelta
from openpyxl import Workbook
from random import sample,randint

'''
Se puede hacer un fichero input.json con los nombres de los médicos y una lista de días de vacaciones 
'''

FESTIVOS = {'01-01','06-01','01-05','02-05','15-05','15-08','12-10','01-11','09-11','06-12','08-12','25-12'}



class turnos:
    def __init__(self, _vacaciones_x_pers, _desde, _hasta,_version=''):
        
        #input
        self.medicos =  set(_vacaciones_x_pers.keys())
        self.vacaciones_x_pers = _vacaciones_x_pers
        
        #para saber cuanto lleva cada doctor si hacer una guardia
        self.todos_han_hecho_guardia = {"doble":False, #fines de semana y festivos
                                    "simple":False}#entre_semana
        self.ultima_guardia_x_pers={"doble":{}, #fines de semana y festivos
                                    "simple":{}}#entre_semana
        
        #para encontrar 4 medicos
        self.festivos = {datetime.strptime(f+'-2019','%d-%m-%Y') for f in FESTIVOS}
        
        #para programar_calendario
        self.programacion={}
        self.inicio = _desde
        self.final = _hasta
        self.dia = _desde
        self._tipo_de_guardia = self._tipo_de_dia()
        
        #para guardar el archivo
        self.version = _version
        
    
    def programar_calendario(self):
        #Para cada día desde el principio hasta el final encontrar 4 medicos        
        while self.dia <= self.final:
            self.programacion[self.dia] = self._encontrar_4_medicos()
            self.dia += timedelta(days=1)  
            self._tipo_de_guardia = self._tipo_de_dia()
    
    def _encontrar_4_medicos(self):
        los_4_medicos=set()                                    
        # Buscar quien está librando ese día
        librando = self._de_vacaciones_o_salientes() #set
        #escoger primero los que todavía no hayan hecho una guardia de este tipo
        if self.todos_han_hecho_guardia[self._tipo_de_guardia]==False:
            los_4_medicos= self._escoger_los_que_todavia_no_han_pasado(librando)
        if len(los_4_medicos)<4:
            los_4_medicos= los_4_medicos.union(self._escoger_medicos_descansados(4 - len(los_4_medicos),
                                                                               librando))
        # apunta las guardias
        for med in los_4_medicos:
            self.ultima_guardia_x_pers[self._tipo_de_guardia][med] = self.dia      
            
        
                                           
    def _de_vacaciones_o_salientes(self, _dia=''):
        dia = self.dia if _dia == '' else _dia
        print(dia)
        #Devolver la lista de medicos que están de vacaciones o salientes        
        vacaciones = {med for med in self.medicos if dia.strftime('%d-%m-%Y') in self.vacaciones_x_pers[med]}
        salientes  = {med for med in self.medicos if self.ultima_guardia_x_pers[self._tipo_de_guardia].get(med) == dia - timedelta(days=1)}
        return set(vacaciones.union(salientes))
    
    def _tipo_de_dia(self):
        #identificar el tipo de guardia
        return 'doble' if self.dia.weekday() in (5,6) or self.dia in self.festivos else 'simple'
    
    def _escoger_los_que_todavia_no_han_pasado(self,_librando):                                 
        los_medicos_que_no_han_pasado=set()
        #medicos disponibles con prioridad: todavía no han hecho este tipo de guardia y que no están de vacaciones
        cola_inicial = self.medicos.difference(set(self.ultima_guardia_x_pers[self._tipo_de_guardia].keys()))
        # Si la cola está vacía es que todos los medicos han hecho una guardia de este tipo ya
        if len(cola_inicial)==0:
            self.todos_han_hecho_guardia[self._tipo_de_guardia]=True
            return los_medicos_que_no_han_pasado #{}
                                       
        disponibles_con_prioridad = cola_inicial.difference(_librando)
        # mientras que no tengas 4 medicos y sobren disponibles añade uno hasta que no queden o que tengas 4
        while len(los_medicos_que_no_han_pasado)<4 and len(disponibles_con_prioridad)>0:
            medico= sample(disponibles_con_prioridad,1)[0]
            los_medicos_que_no_han_pasado.add(medico)
            disponibles_con_prioridad.discard(medico)                                        
        return los_medicos_que_no_han_pasado
                                        
    def _escoger_medicos_descansados(self, numero, librando):
        assert numero in (1,2,3,4)
        medicos_para_elegir= {med for med in self.medicos.difference(librando) 
                              if self.ultima_guardia_x_pers[self._tipo_de_guardia][med]
                              in self._las_guardias_mas_antiguas() }
        medicos_elegidos=set(sample(medicos_para_elegir,numero))
        return medicos_elegidos
        
    def _las_guardias_mas_antiguas(self):
        l=list(self.ultima_guardia_x_pers[self._tipo_de_guardia].values())
        l.sort()
        return l[:4]
    
    def imprimir_calendario(self):
        lista_de_medicos = list(self.medicos)
        wb=Workbook()
        ws=wb.active()
        ws.name='Vacaciones Por Persona'
        ws.cell(row=1,column=1)='Persona';ws.cell(row=1,column=2)='Vacaciones escogidas'
        for num_med in range(lista_de_medicos):            
            ws.cell(row=num_med+2,column=1) =  lista_de_medicos[num_med]
            lista_de_vacaciones=list(self.vacaciones_x_pers[lista_de_medicos[num_med]])
            for num_dia_libre in range(lista_de_vacaciones):
                ws.cell(row=num_med+2,column= num_dia_libre+2) = lista_de_vacaciones[num_dia_libre]
        wb.create_sheet('Calendario')
        ws.cell(row=1,column=1)='Fecha';ws.cell(row=1,column=2)='De guardia';ws.cell(row=1,column=3)='De vacaciones o salientes'
        
        
 
def fecha_random(start, end):
    """Generate a random datetime between `start` and `end`"""
    return (start +timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=randint(0, int((end - start).total_seconds()))
    )).strftime('%d-%m-%y')


def _crear_vacaciones(_desde,_hasta):
    medicos={}
    vacaciones=set()
    for i in range(1,41):
        for d in range(20):
            vacaciones.add(fecha_random(_desde,_hasta))
        medicos["Dra.Eugelilla"+str(i)]=vacaciones 
        vacaciones=set()
    return medicos

if __name__=='__main__':
    desde = input("¿cual es la fecha inicial?")  or '25-09-2019'
    hasta = input("¿cual es la fecha final?") or '25-09-2020'
    desde,hasta =  datetime.strptime(desde,'%d-%m-%Y'), datetime.strptime(hasta,'%d-%m-%Y')
    for i in range(10):
        turnos_del_hospital=turnos(_crear_vacaciones(desde,hasta),desde,hasta,i) 
    turnos_del_hospital.programar_calendario()
    turnos_del_hospital.imprimir_calendario()      

In [16]:
import datetime
d=datetime.datetime.now()
print(d.month)
d.year
print(datetime.strptime(date(2010, 6, 16).strftime('%d-%m-%y')),'%d-%m-%y')
d.__repr__()

9


AttributeError: module 'datetime' has no attribute 'strptime'

In [ ]:
from keyword import kwlist
print(kwlist)


In [78]:
 with open("input1.json",'r') as f:
        medicos_vacaciones = json.load(f) 
        print(medicos_vacaciones.keys())

dict_keys(['Dra.Eugelilla 1', 'Dra.Eugelilla 2', 'Dra.Eugelilla 3', 'Dra.Eugelilla 4', 'Dra.Eugelilla 5', 'Dra.Eugelilla 6', 'Dra.Eugelilla 7', 'Dra.Eugelilla 8', 'Dra.Eugelilla 9', 'Dra.Eugelilla 10', 'Dra.Eugelilla 11', 'Dra.Eugelilla 12', 'Dra.Eugelilla 13', 'Dra.Eugelilla 14', 'Dra.Eugelilla 15', 'Dra.Eugelilla 16', 'Dra.Eugelilla 17', 'Dra.Eugelilla 18', 'Dra.Eugelilla 19', 'Dra.Eugelilla 20', 'Dra.Eugelilla 21', 'Dra.Eugelilla 22', 'Dra.Eugelilla 23', 'Dra.Eugelilla 24', 'Dra.Eugelilla 25', 'Dra.Eugelilla 26', 'Dra.Eugelilla 27', 'Dra.Eugelilla 28', 'Dra.Eugelilla 29', 'Dra.Eugelilla 30', 'Dra.Eugelilla 31', 'Dra.Eugelilla 32', 'Dra.Eugelilla 33', 'Dra.Eugelilla 34', 'Dra.Eugelilla 35', 'Dra.Eugelilla 36', 'Dra.Eugelilla 37', 'Dra.Eugelilla 38', 'Dra.Eugelilla 39', 'Dra.Eugelilla 40'])


In [130]:
class person:
    def __init__(self, name, age):
        self.name=name
        self.age=age
        self.cryptokitties=[]
    def buy_a_cryptokitty(self,cryptokitty_id):
        self.last_cryptokitty_id=cryptokitty_id
        self.cryptokitties.append(last_cryptokitty_id)
    


In [162]:
datetime.datetime.strptime('01-05'+'-2019', '%d-%m-%Y')

datetime.datetime(2019, 5, 1, 0, 0)

In [ ]:
from openpyxl import Workbook
wb=Workbook()
ws=wb.active
